In [2]:
import os

import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

px.set_mapbox_access_token(open("../.mapbox_token").read())

np.random.seed(42)

Results Viz

In [4]:
crime_types = ["THEFT", "BATTERY", "CRIMINAL DAMAGE", "NARCOTICS", "ASSAULT"]
colors = list(sns.color_palette(n_colors = 50).as_hex())

df_locations = pd.read_csv(f"../data/cooked/locations.csv").Location.apply(lambda x: eval(x)).apply(pd.Series).rename(columns = {0: "Latitude", 1: "Longitude"})
df_resource = pd.read_csv(f"../data/cooked/resource_cost.csv", index_col = 0)
df_center = pd.read_csv(f"../data/cooked/dist_from_center.csv")
df_center["cost"] = df_center.Location.apply(lambda x: (x - df_center.Location.min()) * 1_000 + 10_000)

# sample = 1 # up to 4
for sample in range(0, 4 + 1):

    df_distance = pd.read_csv(f"../data/cooked/col_sample_{sample}.csv", index_col = 0, parse_dates=["parsed_date"])
    df_distance["shift"] = df_distance.parsed_date.dt.hour % 8
    df_distance["crime_type"] = df_distance["Primary Type"].apply(lambda x: crime_types.index(x))

    df_z = pd.concat([df_locations, pd.read_csv(f"../data/results/z_{sample + 1}.csv").rename(columns = {"Column1": "is_installation"})], axis = 1)

    # moment = 1 # up to 3
    for moment in range(1, 3 + 1):

        df_y = pd.read_csv(f"../data/results/y/{sample + 1}_{moment}.csv").rename(columns = {f"Column{u}": crime_types[u - 1] for u in range(1, 5 + 1)})
        df_y["covered_sum"] = df_y.sum(axis = 1)

        df_distance_moment = df_distance.copy().reset_index(drop = True)
        df_distance_moment["covered"] = df_y.covered_sum.reset_index(drop = True)
        df_distance_moment["covered_color"] = df_distance_moment.apply(lambda row: row["Primary Type"] if row["covered"] else "Not Covered", axis = 1)

        fig = px.scatter_mapbox(
            df_distance_moment[df_distance_moment.parsed_date.dt.hour % 8 == moment],
            lat = "Latitude",
            lon = "Longitude",
            color = "covered_color",
            color_continuous_scale = px.colors.carto,
            mapbox_style = "basic",
            zoom = 9
        )

        for i, row in df_z.iterrows():

            if not row.is_installation:
                continue

            fig.add_trace(
                go.Scattermapbox(
                    lat = [row.Latitude],
                    lon = [row.Longitude],
                    mode = "markers", 
                    marker = go.scattermapbox.Marker(
                        size = 20,
                        color = colors[i],
                        opacity=0.4,
                    ),
                    name = f"covered_{i}",
                )
            )

        fig.write_image(f"../images/{sample}/{moment}.png")

    break

In [98]:
df_cov_cost = pd.read_csv("../data/results/cov_cost.csv")

df_sample_res = df_cov_cost.Column1.apply(lambda x: eval(x[3:])).apply(pd.Series)
df_sample_res.rename(columns = {0: "max_coverage"}, inplace = True)
df_cov = df_sample_res.apply(lambda col: [c[0] for c in col]).apply(pd.Series)
df_cost = df_sample_res.apply(lambda col: [c[1] for c in col]).apply(pd.Series)

df_cov_cost_res = pd.concat(
    [
        df_cov.melt(var_name = "sample_num", value_name = "coverage"),
        df_cost.melt(var_name = "sample_num_cost", value_name = "cost"),
    ],
    axis = 1
).drop(columns = ["sample_num_cost"])

df_cov_cost_res["coverage"] = df_cov_cost_res["coverage"] / 1_000
df_cov_cost_res["normalized_cost"] = df_cov_cost_res["cost"] / df_cov_cost_res["cost"].max()

fig = px.scatter(
    df_cov_cost_res,
    x = "coverage",
    y = "normalized_cost",
    color = "sample_num",
)

fig.write_image(f"../images/cov_cost.png")

fig.show()

In [99]:
# TODO write elapsed to file in .jl
elapsed = [0.3880000114440918,
 9.141999959945679,
 1946.090000152588,
 2.0179998874664307,
 178.1909999847412,
 1174.180999994278,
 0.40000009536743164,
 0.3659999370574951,
 8.815999984741211,
 69.34700012207031,
 0.9609999656677246,
 112.1930000782013,
 32.16499996185303,
 0.4159998893737793,
 0.3640000820159912,
 2.2280001640319824,
 8.58899998664856,
 102.38800001144409,
 56.218000173568726,
 66.67300009727478,
 0.4099998474121094,
 0.35500001907348633,
 19.11900019645691,
 128.89299988746643,
 102.93299984931946,
 4.309000015258789,
 4.7209999561309814,
 0.5299999713897705,
 0.35199999809265137,
 12.666999816894531,
 3.3440001010894775,
 1.496999979019165,
 84.0019998550415,
 32.37299990653992,
 0.41100001335144043]

elapsed_df = pd.DataFrame({"elapsed": elapsed})

fig = px.histogram(elapsed_df, x = "elapsed", marginal="rug", histnorm="percent")

fig.write_image(f"../images/elapsed.png")

fig.show()